In [1]:
import torch
import numpy
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
import torch.nn.functional as F

c:\users\trans\appdata\local\programs\python\python37\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [5]:
n_dim=2
dim_centers = [[1,1], [-1,-1], [-1,1], [1,-1]]
x_train, y_train = make_blobs(n_samples=80, n_features=n_dim,
                             centers=dim_centers,
                             cluster_std=0.3,
                             shuffle=True)
x_test, y_test = make_blobs(n_samples=20, n_features=n_dim,
                           centers=dim_centers,
                           cluster_std=0.3,
                           shuffle=True)

In [6]:
def label_map(_y, _from, _to):
    y = numpy.copy(_y)
    for f in _from:
        y[_y == f] = _to
    return y